# Source Hackathon - AI for Retail Operations

## Challenge Overview
- **Data**: 2+ years of pharmacy data (20,000+ products, 10+ locations)
- **Problem**: Messy operational data, manual Excel workflows, LLM hallucination risks
- **Goal**: Build AI agents that automate retail operations with proper guardrails

## Dataset Stats
- **Retail Sales**: 1.3M+ transaction lines
- **Online Orders**: 1.1M+ order lines
- **Locations**: 10+ physical stores + Shopify online
- **Time Range**: Sept 2023 - Oct 2025

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load and Explore Retail Sales Data

In [2]:
# Load retail sales - sample first to avoid memory issues
print("Loading retail sales data...")
retail_sales = pd.read_csv('data/input/Retail/retail_sales_data_01_09_2023_to_31_10_2025.csv', 
                           encoding='utf-8-sig',
                           low_memory=False)

print(f"Total rows: {len(retail_sales):,}")
print(f"Columns: {retail_sales.shape[1]}")
print(f"\nColumn names:")
print(retail_sales.columns.tolist())

Loading retail sales data...
Total rows: 1,324,051
Columns: 22

Column names:
['Product', 'Packsize', 'Headoffice ID', 'Barcode', 'OrderList', 'Branch Name', 'Dept Fullname', 'Group Fullname', 'Trade Price', 'RRP', 'Sale Date', 'Sale ID', 'Qty Sold', 'Turnover', 'Vat Amount', 'Sale VAT Rate', 'Turnover ex VAT', 'Disc Amount', 'Discount Band', 'Profit', 'Refund Qty', 'Refund Value']


In [3]:
# Basic info
retail_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324051 entries, 0 to 1324050
Data columns (total 22 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Product          1324051 non-null  object 
 1   Packsize         1312985 non-null  object 
 2   Headoffice ID    1324051 non-null  int64  
 3   Barcode          1316795 non-null  float64
 4   OrderList        1323821 non-null  object 
 5   Branch Name      1324051 non-null  object 
 6   Dept Fullname    1323978 non-null  object 
 7   Group Fullname   1323638 non-null  object 
 8   Trade Price      1324051 non-null  float64
 9   RRP              1324051 non-null  float64
 10  Sale Date        1324051 non-null  object 
 11  Sale ID          1324051 non-null  int64  
 12  Qty Sold         1324051 non-null  int64  
 13  Turnover         1324051 non-null  float64
 14  Vat Amount       1324051 non-null  float64
 15  Sale VAT Rate    1324051 non-null  float64
 16  Turnover ex VAT  1

In [4]:
# First few rows
retail_sales.head(10)

,Product,Packsize,Headoffice ID,Barcode,OrderList,Branch Name,Dept Fullname,Group Fullname,Trade Price,RRP,Sale Date,Sale ID,Qty Sold,Turnover,Vat Amount,Sale VAT Rate,Turnover ex VAT,Disc Amount,Discount Band,Profit,Refund Qty,Refund Value
0,Revive Active Tropical Health Food Supplement ...,30,93754,7.947128e+11,Galway Natural Health Company,Baggot St,Vitamins,Revive Active,26.59,59.95,2023-09-01 08:05:00,6219657,1,44.96,5.35,13.5,39.61,14.99,NaN,8.69,0,0.0
1,Catrice True Skin High Cover Concealer 010,0,90866,4.059729e+12,SCL Catrice Make Up,Baggot St,Self Selection Stands,Catrice Make Up,2.53,4.65,2023-09-01 08:13:00,6219658,1,4.95,0.93,23.0,4.02,0.00,NaN,1.32,0,0.0
2,Durex Thin Feel 6s,6's,47131,5.052197e+12,Reckitt Benckiser Group,Baggot St,OTC : Family Planning,Family Planning,4.20,8.50,2023-09-01 08:25:00,6219659,1,6.00,0.71,13.5,5.29,0.00,NaN,2.33,0,0.0
3,Riley Applicator Tampons Regular 12 Pack,12,95403,6.309410e+11,We Are Riley,Baggot St,Female Toiletries : Hygiene,Female Hygiene,3.70,5.29,2023-09-01 08:27:00,6219660,1,5.29,0.00,0.0,5.29,0.00,NaN,1.59,0,0.0
4,Elave Daily Skin Defence SPF45 50ml,50ml,17710,5.098928e+12,Gardiner Apothecary,Baggot St,Skincare,Skin Care,10.89,22.95,2023-09-01 08:28:00,6219661,1,22.95,4.29,23.0,18.66,0.00,NaN,7.77,0,0.0
5,Linwoods Flax Seed 425G,425g,14284,5.016887e+12,Wholefoods Wholesale,Baggot St,Nutritional Supplements : Diet,Nutritional Supplements,5.30,7.99,2023-09-01 08:29:00,6219662,1,7.99,0.00,0.0,7.99,0.00,NaN,2.69,0,0.0
6,Solpadeine Soluble 24s,24s,2371,5.010215e+12,Pharmax,Baggot St,OTC : Analgesics,Analgesics,8.36,14.75,2023-09-01 08:29:00,6219662,1,13.00,0.00,0.0,13.00,0.00,NaN,7.12,0,0.0
7,MacuSave Eye Food Supplement 30s,30,76204,5.060135e+12,Wholefoods Wholesale,Baggot St,Vitamins,Other Vitamins,10.60,20.05,2023-09-01 08:32:00,6219663,1,19.39,0.00,0.0,19.39,0.00,NaN,7.61,0,0.0
8,Actifed Tabs 12s,12s,2271,5.010124e+12,Kenvue (McNeil Healthcare),Baggot St,OTC : Cold & Flu,Cough/Cold/Flu,4.25,8.95,2023-09-01 08:33:00,6219664,1,7.80,0.00,0.0,7.80,0.00,NaN,3.24,0,0.0
9,Medicare Alcohol Free Cleansing Wipes 10,10,89689,5.099390e+12,Fleming Medical,Baggot St,OTC : First Aid,First aid,1.74,3.80,2023-09-01 08:33:00,6219665,1,3.45,0.65,23.0,2.80,0.00,NaN,1.04,0,0.0


In [5]:
# Convert date column
retail_sales['Sale Date'] = pd.to_datetime(retail_sales['Sale Date'])

# Basic stats
print("Date Range:", retail_sales['Sale Date'].min(), "to", retail_sales['Sale Date'].max())
print(f"\nUnique Products: {retail_sales['Product'].nunique():,}")
print(f"Unique Branches: {retail_sales['Branch Name'].nunique()}")
print(f"Unique Departments: {retail_sales['Dept Fullname'].nunique()}")
print(f"\nBranches:")
print(retail_sales['Branch Name'].value_counts())

ValueError: time data "31/10/2025 19:01" doesn't match format "%Y-%m-%d %H:%M:%S", at position 367137. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

## 2. Data Quality Analysis - Finding the Mess!

In [ ]:
# Missing data analysis
print("Missing Data Analysis:")
print("="*50)
missing = retail_sales.isnull().sum()
missing_pct = 100 * missing / len(retail_sales)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
print(missing_df)

In [ ]:
# Check for duplicate product entries with different names
print("\nDuplicate Detection - Same Barcode, Different Names:")
print("="*50)
barcode_groups = retail_sales.groupby('Barcode')['Product'].unique()
duplicates = barcode_groups[barcode_groups.apply(len) > 1]
print(f"Found {len(duplicates)} barcodes with multiple product names!")
print("\nExamples:")
for barcode, names in list(duplicates.items())[:5]:
    print(f"Barcode {barcode}:")
    for name in names:
        print(f"  - {name}")
    print()

In [ ]:
# Pricing anomalies - selling below cost
print("\nPricing Anomalies - Negative Margins:")
print("="*50)
negative_margin = retail_sales[retail_sales['Profit'] < 0]
print(f"Transactions with negative profit: {len(negative_margin):,} ({100*len(negative_margin)/len(retail_sales):.2f}%)")
print(f"Total loss from negative margins: €{negative_margin['Profit'].sum():,.2f}")
print("\nTop loss-making products:")
negative_by_product = negative_margin.groupby('Product').agg({
    'Profit': 'sum',
    'Qty Sold': 'sum',
    'Turnover': 'sum'
}).sort_values('Profit')
print(negative_by_product.head(10))

In [ ]:
# Heavy discount analysis
print("\nHeavy Discounting Analysis:")
print("="*50)
retail_sales['Discount %'] = 100 * retail_sales['Disc Amount'] / (retail_sales['Turnover'] + retail_sales['Disc Amount'])
heavy_discount = retail_sales[retail_sales['Discount %'] > 20]
print(f"Transactions with >20% discount: {len(heavy_discount):,} ({100*len(heavy_discount)/len(retail_sales):.2f}%)")
print(f"Total discount given: €{retail_sales['Disc Amount'].sum():,.2f}")
print(f"\nAverage discount rate: {retail_sales['Discount %'].mean():.2f}%")
print(f"Products with highest average discount:")
discount_by_product = retail_sales.groupby('Product').agg({
    'Discount %': 'mean',
    'Disc Amount': 'sum',
    'Qty Sold': 'sum'
}).sort_values('Discount %', ascending=False)
print(discount_by_product.head(10))

## 3. Sales Performance Analysis

In [ ]:
# Top performing products
print("Top 20 Products by Revenue:")
print("="*50)
top_products = retail_sales.groupby('Product').agg({
    'Turnover': 'sum',
    'Profit': 'sum',
    'Qty Sold': 'sum',
    'Trade Price': 'mean',
    'RRP': 'mean'
}).sort_values('Turnover', ascending=False)

top_products['Margin %'] = 100 * top_products['Profit'] / top_products['Turnover']
top_products['Avg Selling Price'] = top_products['Turnover'] / top_products['Qty Sold']
print(top_products.head(20))

In [ ]:
# Sales by location
print("\nSales by Branch:")
print("="*50)
branch_performance = retail_sales.groupby('Branch Name').agg({
    'Turnover': 'sum',
    'Profit': 'sum',
    'Qty Sold': 'sum',
    'Sale ID': 'nunique'
}).sort_values('Turnover', ascending=False)
branch_performance.columns = ['Revenue €', 'Profit €', 'Units Sold', 'Transactions']
branch_performance['Margin %'] = 100 * branch_performance['Profit €'] / branch_performance['Revenue €']
branch_performance['Avg Transaction €'] = branch_performance['Revenue €'] / branch_performance['Transactions']
print(branch_performance)

In [ ]:
# Sales trends over time
daily_sales = retail_sales.groupby(retail_sales['Sale Date'].dt.date).agg({
    'Turnover': 'sum',
    'Profit': 'sum',
    'Sale ID': 'nunique'
})

fig, axes = plt.subplots(3, 1, figsize=(14, 10))

daily_sales['Turnover'].plot(ax=axes[0], title='Daily Revenue Over Time', color='green', alpha=0.7)
axes[0].set_ylabel('Revenue €')
axes[0].axhline(daily_sales['Turnover'].mean(), color='red', linestyle='--', alpha=0.5, label='Average')
axes[0].legend()

daily_sales['Profit'].plot(ax=axes[1], title='Daily Profit Over Time', color='blue', alpha=0.7)
axes[1].set_ylabel('Profit €')
axes[1].axhline(daily_sales['Profit'].mean(), color='red', linestyle='--', alpha=0.5, label='Average')
axes[1].legend()

daily_sales['Sale ID'].plot(ax=axes[2], title='Daily Transactions Over Time', color='purple', alpha=0.7)
axes[2].set_ylabel('Transactions')
axes[2].axhline(daily_sales['Sale ID'].mean(), color='red', linestyle='--', alpha=0.5, label='Average')
axes[2].legend()

plt.tight_layout()
plt.show()

print(f"Average daily revenue: €{daily_sales['Turnover'].mean():,.2f}")
print(f"Average daily profit: €{daily_sales['Profit'].mean():,.2f}")
print(f"Average daily transactions: {daily_sales['Sale ID'].mean():,.0f}")

## 4. Load Inventory Snapshot

In [ ]:
# Load inventory snapshot
print("Loading inventory snapshot...")
inventory = pd.read_csv('data/input/Retail/retail_inventory_snapshot_30_10_25.csv', 
                        encoding='utf-8-sig',
                        low_memory=False)

print(f"Total rows: {len(inventory):,}")
print(f"Unique products: {inventory['Product'].nunique():,}")
inventory.head()

In [ ]:
# Total inventory value by branch
inventory_summary = inventory.groupby('Branch Name').agg({
    'Branch Stock Level': 'sum',
    'Product': 'count'
})
inventory_summary.columns = ['Total Units', 'SKU Count']

# Calculate value
inventory['Stock Value'] = inventory['Branch Stock Level'] * inventory['Trade Price']
inventory_value = inventory.groupby('Branch Name')['Stock Value'].sum().sort_values(ascending=False)

print("Inventory by Branch:")
print("="*50)
inv_df = pd.DataFrame({
    'Total Units': inventory_summary['Total Units'],
    'SKU Count': inventory_summary['SKU Count'],
    'Stock Value €': inventory_value
})
print(inv_df.sort_values('Stock Value €', ascending=False))
print(f"\nTotal inventory value: €{inventory['Stock Value'].sum():,.2f}")

## 5. Inventory Optimization Analysis

In [ ]:
# Calculate sales velocity for last 30 days
recent_sales = retail_sales[retail_sales['Sale Date'] >= retail_sales['Sale Date'].max() - pd.Timedelta(days=30)]

# Aggregate by product
velocity = recent_sales.groupby('Product').agg({
    'Qty Sold': 'sum',
    'Turnover': 'sum',
    'Profit': 'sum'
})

velocity.columns = ['Units_30d', 'Revenue_30d', 'Profit_30d']
velocity['Daily_Velocity'] = velocity['Units_30d'] / 30

# Merge with current stock
inventory_agg = inventory.groupby('Product').agg({
    'Branch Stock Level': 'sum',
    'Trade Price': 'first',
    'RRP': 'first'
})
inventory_agg.columns = ['Current_Stock', 'Cost', 'RRP']

# Combine
stock_analysis = inventory_agg.join(velocity, how='left').fillna(0)
stock_analysis['Days_of_Stock'] = np.where(
    stock_analysis['Daily_Velocity'] > 0,
    stock_analysis['Current_Stock'] / stock_analysis['Daily_Velocity'],
    999
)
stock_analysis['Stock_Value'] = stock_analysis['Current_Stock'] * stock_analysis['Cost']

print("Stock Coverage Analysis:")
print("="*50)
print(f"Products with <7 days stock (URGENT): {len(stock_analysis[stock_analysis['Days_of_Stock'] < 7]):,}")
print(f"Products with 7-21 days stock (OPTIMAL): {len(stock_analysis[(stock_analysis['Days_of_Stock'] >= 7) & (stock_analysis['Days_of_Stock'] <= 21)]):,}")
print(f"Products with >60 days stock (SLOW MOVERS): {len(stock_analysis[stock_analysis['Days_of_Stock'] > 60]):,}")
print(f"Products with no sales in last 30 days: {len(stock_analysis[stock_analysis['Units_30d'] == 0]):,}")

In [ ]:
# Urgent reorders needed
print("\nURGENT REORDERS NEEDED (<7 days stock):")
print("="*50)
urgent = stock_analysis[
    (stock_analysis['Days_of_Stock'] < 7) & 
    (stock_analysis['Daily_Velocity'] > 0)
].sort_values('Revenue_30d', ascending=False).head(20)

urgent_display = urgent[['Current_Stock', 'Daily_Velocity', 'Days_of_Stock', 'Revenue_30d', 'Profit_30d']]
urgent_display.columns = ['Stock', 'Daily Sales', 'Days Left', 'Revenue 30d', 'Profit 30d']
print(urgent_display)

In [ ]:
# Slow movers - dead stock
print("\nSLOW MOVERS - MARKDOWN CANDIDATES (>60 days stock):")
print("="*50)
slow_movers = stock_analysis[
    (stock_analysis['Days_of_Stock'] > 60) & 
    (stock_analysis['Current_Stock'] > 0)
].sort_values('Stock_Value', ascending=False).head(20)

slow_display = slow_movers[['Current_Stock', 'Daily_Velocity', 'Days_of_Stock', 'Stock_Value', 'Revenue_30d']]
slow_display.columns = ['Stock', 'Daily Sales', 'Days Stock', 'Locked Capital €', 'Revenue 30d']
print(slow_display)
print(f"\nTotal capital locked in slow movers: €{slow_movers['Stock_Value'].sum():,.2f}")

## 6. Department Performance

In [ ]:
# Department analysis
dept_performance = retail_sales.groupby('Dept Fullname').agg({
    'Turnover': 'sum',
    'Profit': 'sum',
    'Qty Sold': 'sum',
    'Sale ID': 'nunique'
}).sort_values('Turnover', ascending=False)

dept_performance['Margin %'] = 100 * dept_performance['Profit'] / dept_performance['Turnover']
dept_performance.columns = ['Revenue €', 'Profit €', 'Units', 'Transactions', 'Margin %']

print("Top 20 Departments by Revenue:")
print("="*50)
print(dept_performance.head(20))

# Visualize top 10
top_10_depts = dept_performance.head(10)
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

top_10_depts['Revenue €'].plot(kind='barh', ax=axes[0], color='green')
axes[0].set_title('Top 10 Departments by Revenue')
axes[0].set_xlabel('Revenue €')

top_10_depts['Margin %'].plot(kind='barh', ax=axes[1], color='blue')
axes[1].set_title('Top 10 Departments - Margin %')
axes[1].set_xlabel('Margin %')

plt.tight_layout()
plt.show()

## 7. Key Insights Summary

In [ ]:
print("="*70)
print("KEY INSIGHTS & HACKATHON OPPORTUNITIES")
print("="*70)

print("\n1. DATA QUALITY ISSUES (Perfect for Data-Fix Agent):")
print(f"   - {len(duplicates)} barcodes have multiple product names")
print(f"   - {len(negative_margin):,} transactions with negative margins (€{negative_margin['Profit'].sum():,.2f} loss)")
print(f"   - {len(heavy_discount):,} transactions heavily discounted (>20%)")
print(f"   - Missing data in multiple columns")

print("\n2. INVENTORY OPTIMIZATION OPPORTUNITIES:")
print(f"   - {len(stock_analysis[stock_analysis['Days_of_Stock'] < 7]):,} products need URGENT reorder")
print(f"   - {len(stock_analysis[stock_analysis['Days_of_Stock'] > 60]):,} slow-moving products")
print(f"   - €{slow_movers['Stock_Value'].sum():,.2f} locked in slow-moving inventory")
print(f"   - Products with no sales: {len(stock_analysis[stock_analysis['Units_30d'] == 0]):,}")

print("\n3. REVENUE & PROFITABILITY:")
print(f"   - Total revenue (2 years): €{retail_sales['Turnover'].sum():,.2f}")
print(f"   - Total profit: €{retail_sales['Profit'].sum():,.2f}")
print(f"   - Average margin: {100*retail_sales['Profit'].sum()/retail_sales['Turnover'].sum():.2f}%")
print(f"   - Total discounts given: €{retail_sales['Disc Amount'].sum():,.2f}")

print("\n4. OPERATIONS SCALE:")
print(f"   - Unique products: {retail_sales['Product'].nunique():,}")
print(f"   - Total transactions: {retail_sales['Sale ID'].nunique():,}")
print(f"   - Locations: {retail_sales['Branch Name'].nunique()}")
print(f"   - Departments: {retail_sales['Dept Fullname'].nunique()}")

print("\n" + "="*70)

## 8. Sample Online Data (Shopify)

In [ ]:
# Sample online orders (just first 100k rows to avoid memory issues)
print("Loading online orders sample...")
online_orders = pd.read_csv('data/input/Online/order_line_items.csv', 
                            encoding='utf-8-sig',
                            nrows=100000,
                            low_memory=False)

print(f"Sample rows: {len(online_orders):,}")
print(f"Columns: {online_orders.shape[1]}")
online_orders.head()

In [ ]:
# Online products
print("\nLoading Shopify products...")
online_products = pd.read_csv('data/input/Online/inventory_products.csv', 
                              encoding='utf-8-sig',
                              low_memory=False)

print(f"Online products: {len(online_products):,}")
online_products.head()